In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import ScratchLSTM as lstm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional


import importlib
import ScratchLSTM
import ScratchEmbedding
import ScratchRNN
import ScratchDense
import ScratchBiLSTM
import Model

importlib.reload(Model)
importlib.reload(ScratchEmbedding)
importlib.reload(ScratchDense)
importlib.reload(ScratchLSTM)
importlib.reload(ScratchRNN)
importlib.reload(ScratchBiLSTM)

<module 'ScratchBiLSTM' from 'f:\\Repository\\Semester 6\\cnn-rnn-lstm-scratch-ml-tubes-2\\lstm\\ScratchBiLSTM.py'>

In [148]:
path = "../nusax/datasets/sentiment/indonesian/"

train_data = pd.read_csv(path+"train.csv")
test_data = pd.read_csv(path+"test.csv")

# Text extraction
X_train_text = train_data['text'].astype(str).values
y_train = train_data['label'].values

X_test_text = test_data['text'].astype(str).values
y_test = test_data['label'].values

# Encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Configure tokenizer
vocab_size = 2796
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

# Tokenization
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

# Padding
max_len = 77
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

In [156]:
model = load_model('RNN/model_type_2_RNN.keras')
new_model = Model.convertModel(model, model_name="CNN")

model.summary()

LSTM Layer flags: {}
RNN Layer flags: {}
0 Embedding layer found
1 BiRNN layer found
3 Dense layer found
DenseScratch initialized with weights shape: 256
4 Dense layer found
DenseScratch initialized with weights shape: 64


Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_26 (Embedding)        │ (None, 77, 128)        │       357,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,320,971 (5.04 MB)

 Trainable params: 440,323 (1.68 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 880,648 (3.36 MB)

In [157]:
# Prediction usiong scratch model
y_pred_scratch = []
for x in X_test_pad:
    y_pred_probs = new_model.predict(x.reshape(1, -1))
    y_pred_scratch.append(np.argmax(y_pred_probs, axis=1))

f1 = f1_score(y_test_encoded, y_pred_scratch, average='macro')
print(f"F1 Score Scratch Model: {f1}")

# Prediction using Keras model
y_pred_keras = model.predict(X_test_pad)
y_pred_keras_classes = np.argmax(y_pred_keras, axis=1)
f1_keras = f1_score(y_test_encoded, y_pred_keras_classes, average='macro')
print(f"F1 Score Keras Model: {f1_keras}")

F1 Score Scratch Model: 0.5596044118211898
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
F1 Score Keras Model: 0.5596044118211898
